#**PySpark: DataFrames**




In [ ]:
# Clone the GitHub repository
!git clone https://github.com/ssalloum/SDSC-Spark5.git

In [ ]:
!ls /content/SDSC-Spark5/data

# PySpark
PySpark is an interface for Apache Spark that allows users to write Spark applications using python APIs. PySpark supports most of Spark’s features such as Spark SQL, Streaming, MLlib (Machine Learning) and Spark Core. For detailed information on these components and APIs, please refer to the [official PySpark Documentation](https://spark.apache.org/docs/latest/api/python/index.html).

In [ ]:
#You don't need this on Databricks
!pip install pyspark

## Spark SQL
* [Spark SQL](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/index.html) is a Spark module for structured data processing.
* Spark SQL integrates relational processing (using SQL) and functional programming (using the DataFrame API).



## SparkSession
* An essentail class in Spark SQL is [pyspark.sql.SparkSession](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.SparkSession.html) which represents a unified entry point to programming in Spark.

* In Spark-shell or Databricks notebooks, a SparkSession is created for you, stored in a variable called `spark`.

In [ ]:
# You don't need this on Databricks or spark-shell
from pyspark.sql import SparkSession

# Create a Spark Session
spark = SparkSession.builder\
        .master("local[*]")\
        .appName("Intro to PySpark")\
        .getOrCreate()

In [ ]:
# Check Spark Session Information
spark

## SparkContext
* [SparkContext](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.SparkContext.html) was the main entry point in earlier versions of Spark.
* For working with low-level APIs, [Resilient Distributed Datasets (RDDs)](https://spark.apache.org/docs/latest/rdd-programming-guide.html), and for backward compatibility, you can access SparkContext via SparkSession.

In [ ]:
# get SparkContext
sc = spark.sparkContext
sc

# Spark DataFrames


*   [pyspark.sql.DataFrame](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.html) represents a distributed collection of data grouped into named columns.

* [pyspark.sql.Column](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.Column.html): represents a column in a DataFrame.
* [pyspark.sql.Row](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.Row.html): represents a row in a DataFrame.
*   [pyspark.sql.functions](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/functions.html): common functions to work with DataFrames.

* A DataFrame can be constructed from a variety of [supported data sources](https://spark.apache.org/docs/latest/sql-data-sources.html).


## DataFrameReader
* [DataFrameReader](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrameReader.html): loading DataFrames from external sources.
* You cannot create an instance of DataFrameReader.
* You can access a DataFrameReader through a SparkSession instance using the [read](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.SparkSession.read.html) property to read data from a static data source (streaming data sources has a different method: readStream).
* DataFrameReader provides several public methods that can be used with all supported data sources, and may take different arguments for each source: [format](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrameReader.format.html), [option](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrameReader.option.html), [schema](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrameReader.schema.html), and [load](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrameReader.load.html).

* If you don’t specify the format, then the default is
Parquet or whatever is set in 'spark.sql.sources.default'.

* DataFrameReader also has methods to directly load data from specific formats/sources such as [parquet](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrameReader.parquet.html), [csv](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrameReader.csv.html), [json](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrameReader.json.html).

### Creating DataFrames From CSV files
* You can read data from a [CSV file](https://spark.apache.org/docs/latest/sql-data-sources-csv.html) into a DataFrame.
* The [pyspark.sql.SparkSession.read](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.SparkSession.read.html) function can be used to read in the CSV file and returns a DataFrame of rows and named columns with the types dictated in the schema. We will use csv files from the flights dataset:

In [ ]:
dataPath = "/content/SDSC-Spark5/data/2015-summary.csv"

In [ ]:
#try with inferSchema
flights_df = spark.read\
  .option("inferSchema", "true")\
  .option("header", "true")\
  .csv(dataPath)

In [ ]:
flights_df.show(5)

In [ ]:
flights_df.count()

In [ ]:
flights_df.printSchema()

## DataFrame Schema
* A schema in Spark defines the column names and associated data types for a DataFrame. In addition to inferring the schema from the source data, Spark allows you to define a schema programmatically.

* A schema is a [StructType](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.types.StructType.html) made up of a number of fields, each field is a [StructField](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.types.StructField.htm), that have a name, type, a Boolean flag which specifies whether that column can contain missing or null values, and, finally, users can optionally specify associated metadata with that column.
*  Supported data types are defined in [pyspark.sql.types](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/data_types.html).

* [Spark SQL Guide: Data Types](https://spark.apache.org/docs/latest/sql-ref-datatypes.html)

In [ ]:
#Define a schema programatically
from pyspark.sql.types import *

myFlightSchema = StructType([
  StructField("dest", StringType(), True),
  StructField("origin", StringType(), True),
  StructField("flights", LongType(), False)
])

myFlightSchema

In [ ]:
flights_df_2015 = spark.read.schema(myFlightSchema).option("header", "true").csv(dataPath)

In [ ]:
flights_df_2015.take(5)

## Columns
* [DataFrame.columns](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.columns.html): get all columns names in a DataFrame.



In [ ]:
flights_df_2015.columns

* you can refer to columns in a number of different
ways; and you can use them interchangeably: [col()](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.col.html), [column()](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.column.html), [expr()](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.expr.html).

In [ ]:
from pyspark.sql.functions import expr, col, column

flights_df_2015.select(
  col("dest"),
  column("dest"),
  expr("lower(dest)"),
  flights_df_2015.dest)\
.show(5)

* In Spark DataFrames, Columns are objects represented by [pyspark.sql.Column](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.Column.html) that provides commonly used methods on columns.

In [ ]:
from pyspark.sql import Column

flights_df_2015.orderBy(flights_df_2015.flights.desc()).show(5)

## Rows
* A row in Spark is an object of [pyspark.sql.Row](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.Row.html), containing one or more columns.


In [ ]:
#get the first Row
flights_df_2015.first()

In [ ]:
#get a list of the first "num" of Rows

In [ ]:
flights_df_2015.take(5)

* Because Row is an object in Spark and an ordered collection of fields, you can instantiate a Row in each of Spark’s supported languages and access its fields by an index starting at 0:

In [ ]:
from pyspark.sql import Row

blog_row = Row(6, "Reynold", "Xin", "https://tinyurl.6", 255568, "3/2/2015",
["twitter", "LinkedIn"])

# access using index for individual items
blog_row[1]
'Reynold'

In [ ]:
# the following code results in an array of Row objects.
spark.range(5).show()

Row objects can be used to create DataFrames if you need them for quick interactivity
and exploration:

In [ ]:
rows = [Row("Matei Zaharia", "CA"), Row("Reynold Xin", "CA")]
authors_df = spark.createDataFrame(rows, ["Authors", "State"])
authors_df.show()
authors_df.printSchema()

## Parquet Data Source
* [Parquet](https://parquet.apache.org/) is an open-source columnar format that offers many I/O
optimizations (such as compression, which saves storage space and allows for quick
access to data columns).

* [Parquet files](https://github.com/apache/parquet-format#file-format) are stored in a directory structure that contains the data files, metadata,
a number of compressed files, and some status files.

* Spark SQL provides support for [reading and writing Parquet files](https://spark.apache.org/docs/latest/sql-data-sources-parquet.html).

* Parquet is the default data
source in Spark.

* Unless you are reading from a streaming data source, there’s no need to supply a
schema when reading from a Parquet file, because Parquet saves it as part of its metadata.

* Another way to read this same data using the [parquet](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrameReader.parquet.html) method.

In [ ]:
parquetPath = "/content/SDSC-Spark5/data/2010-summary.parquet"

In [ ]:
df2 = spark.read.parquet(parquetPath)

In [ ]:
df2.printSchema()

## DataFrameWriter
* [DataFrameWriter](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrameWriter.html) is an interface used to write a DataFrame to external stoage systems.

* Unlike with DataFrameReader, you access its instance not from a SparkSession but from the DataFrame you wish to save.

* To get an instance handle, use the [DataFrame.write](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.write.html) method for static data sources (DataFrame.writeStream for streaming data sources).

* It also provides several public methods: [format](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrameWriter.format.html), [option](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrameWriter.option.html), [bucketBy](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrameWriter.bucketBy.html), [save](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrameWriter.save.html), and [saveAsTable](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrameWriter.saveAsTable.html).
* DataFrameWriter also has methods to directly write data to specific formats/sources such as [parquet](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrameWriter.parquet.html), [csv](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrameWriter.csv.html), [json](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrameWriter.json.html).

In [ ]:
df2.write.parquet(path="/tmp/data/df_parquet1",
  mode="overwrite",
  compression="snappy")

#DataFrame Operations: Transformations and Actions

* Spark operations on DataFrames can be classified into two types: transformations and actions.
* All transformations are evaluated lazily. Their results are not computed immediately,
but they are recorded as a lineage. This allows Spark to optimize the execution
plan.
* Distributed computation occurs upon invoking an action on a DataFrame, e.g.,: `show(), take(), count(), collect()`.

### select
The easiest way to work with columns is just to use the [select](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.select.html) method and pass in the column names as strings:

In [ ]:
flights_df.select("DEST_COUNTRY_NAME").show(2)

In [ ]:
flights_df.select("DEST_COUNTRY_NAME","ORIGIN_COUNTRY_NAME").show(2)

### Adding columns
To add a new column to your DataFrame, you can use the [withColumn](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.withColumn.html) method:

In [ ]:
flights_df.withColumn("withinCountry", expr("ORIGIN_COUNTRY_NAME == DEST_COUNTRY_NAME"))\
.take(5)

### Renaming columns
You can rename a column  with the [withColumnRenamed](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.withColumnRenamed.html) method:

In [ ]:
flights_df.withColumnRenamed("DEST_COUNTRY_NAME", "Destination").show(2)

In [ ]:
# renaming multiple columns
flights_df.withColumnRenamed("DEST_COUNTRY_NAME", "dest")\
  .withColumnRenamed("ORIGIN_COUNTRY_NAME", "origin").show(2)

### Removing columns
[drop](https://spark.apache.org/docs/latest/api/python/reference/pyspark.pandas/api/pyspark.pandas.DataFrame.drop.html) is a dedicated method to remove columns from a DataFrame.

In [ ]:
flights_df.drop("count").columns

In [ ]:
flights_df.drop("ORIGIN_COUNTRY_NAME","count").show(5)

### Filtering Rows
There are two methods to perform filtering operations: you can use [where](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.where.html) or [filter](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.filter.html)
and they both will perform the same operation and accept the same argument types when used
with DataFrames. To filter rows, you need an expression that evaluates to true or false.

In [ ]:
flights_df.filter(col("count") < 2).show(2)

In [ ]:
flights_df.where("count < 2").show(2)

In [ ]:
flights_df.where(col("count") < 2).where(col("ORIGIN_COUNTRY_NAME") != "Singapore")\
.show(2)

### Getting Unique Rows
To extract the unique or distinct values in a DataFrame, you can use the [distinct](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.distinct.html?highlight=distinct#pyspark.sql.DataFrame.distinct) method on a
DataFrame.

In [ ]:
flights_df.select("ORIGIN_COUNTRY_NAME", "DEST_COUNTRY_NAME").distinct().count()

In [ ]:
flights_df.select("ORIGIN_COUNTRY_NAME").distinct().count()

### Random Samples
To sample some random records from your DataFrame, use the [sample](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.sample.html) method.

In [ ]:
flights_df.sample(withReplacement = False,
                      fraction= 0.5,
                      seed = 5).count()

### Sorting Rows

There are two equivalent operations to sort the values in a DataFrame: [sort](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.sort.html) and [orderBy](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.orderBy.html).

In [ ]:
flights_df.sort("count", ascending=False).show(5)

In [ ]:
flights_df.orderBy("count", "DEST_COUNTRY_NAME").show(5)

In [ ]:
flights_df.orderBy(col("count"), col("DEST_COUNTRY_NAME"), ascending=False).show(5)

Let’s find the top five destination countries in the data.

In [ ]:
from pyspark.sql.functions import desc

flights_df\
  .groupBy("DEST_COUNTRY_NAME")\
  .sum("count")\
  .withColumnRenamed("sum(count)", "destination_total")\
  .sort(desc("destination_total"))\
  .limit(5)\
  .show()